In [1]:
import os
import numpy as np
import pandas as pd
import random
from tqdm import tqdm
from Bio import SeqIO
import pickle as pkl
from Bio import SeqIO
from Bio.Seq import Seq

# HiC data and ATAC-seq data

In [2]:
resolution = int(1e6)
imgaic_np = pkl.load(open(f'data/imagic_array_{resolution}.bin','rb'))
atac_np = pkl.load(open(f'data/atacseq_list_{resolution}.bin','rb'))
hic_np = pkl.load(open(f'data/hic_array_{resolution}.bin','rb'))

name2idx = pkl.load(open(f'name2idx_{resolution}.bin','rb'))

# Extract sequence information

In [3]:
refdir="ref/"
reffile = 'GRCh38_no_alt_analysis_set_GCA_000001405.15.fasta'

In [4]:

chrom2seq = {}
for record in SeqIO.parse(refdir+reffile,'fasta'):
    try:
        chrom = str(record.id)
        seq = str(record.seq)
        chrom2seq[chrom] = seq
    except Exception as e:
        print(e)
        continue

# Extract all sequences

In [5]:
datadir='data/iMAGIC/data/'
filename = 'GSM4006840_HUVEC_control_iMARGI_inter_sampled.bedpe'

In [6]:
seq = chrom2seq['chr4']
def comp(s,strand):
    if strand=="-":
        return str(Seq(s).complement())
    else:
        return s

In [7]:
def lcs(s1, s2):
    result = 0
    c = np.zeros((len(s1) + 1, len(s2) + 1))
    sim_c = np.zeros((len(s1) + 1, len(s2) + 1))
    for i in range(1, len(s1) + 1):
        for j in range(1, len(s2) + 1):
            if (s1[i - 1] == s2[j - 1]):
                c[i][j] = c[i - 1][j - 1] + 1
                #sim_c[i][j] = sim_c[i - 1][j - 1] + 1
                result = max(result, c[i][j])
            else:
                #sim_c[i][j] = max(sim_c[i][j - 1], sim_c[i - 1][j])
                c[i][j] = 0
    return result#, sim_c[len(s1)][len(s2)]

In [9]:
outputdir='iMAGIC/data/'

input_length=50
def comp(s):
    return str(Seq(s).complement())
with open(outputdir+f'GSM4006840_HUVEC_control_iMARGI_inter_{resolution}_sampled.seqs','w') as fp:
    for line in open(datadir+filename):
        try:
            lines = line.split()
            chr1 = lines[0]
            start1 = int(lines[1])
            end1 = int(lines[2])
            mid1 = (start1 + end1)//2
            chr2 = lines[3]
            start2 = int(lines[4])
            end2 = int(lines[5])
            mid2 = (start2 + end2)//2
            s = lines[-1]
            t = lines[-2]
            seq1 = chrom2seq[chr1][mid1-50:mid1+51]
            seq2 =chrom2seq[chr2][mid2-50:mid2+51]


            chr1 = int(chr1[3:])
            chr2 = int(chr2[3:])
            pos1 = int(mid1 // resolution + 1)
            pos2 = int(mid2 // resolution + 1)
            name1 = f'chr{chr1:02d}_{pos1:04d}'
            name2 = f'chr{chr2:02d}_{pos2:04d}'

            idx1 = name2idx[name1]
            idx2 = name2idx[name2]

            imagi_count = imgaic_np[idx1][idx2]
            hic_count = hic_np[idx1][idx2]

            atac1 = atac_np[idx1][0]
            atac2 = atac_np[idx2][0]
            fp.write(f'{seq1}\t{seq2}\t{chr1}\t{start1}\t{end1}\t{idx1}\t{chr2}\t{start2}\t{end2}\t{idx2}\t{imagi_count}\t{hic_count}\t{atac1}\t{atac2}\n')
        except Exception as e:
            pass

# Generate training data

In [14]:
import random
import numpy as np
np.random.seed(65535)
outputdir='data/iMAGIC/data/'
inputdata = f'{outputdir}/GSM4006840_HUVEC_control_iMARGI_inter_{resolution}.seqs'

rnas,dnas = [],[]
pospairs = []
for line in open(inputdata):
    lines = line.strip().split()
    try:
        rna = lines[0]
        dna = lines[1]
        rnas.append(rna)
        dnas.append(dna)
        pospairs.append((rna,dna))
    except:
        continue

In [10]:
# New mode
import random
import numpy as np
np.random.seed(65535)
resolution = int(1e6)
outputdir='data/iMAGIC/data/'
# Only sample a subset from all positive samples
inputdata = f'{outputdir}/GSM4006840_HUVEC_control_iMARGI_inter_{resolution}_sampled.seqs'

with open(f'{outputdir}/GSM4006840_HUVEC_control_iMARGI_inter_{resolution}_neg_sampled.seqs','w') as fp:
    for line in open(inputdata):

        lines = line.strip().split()
        seq1 = lines[0]
        chr1 = lines[2]
        start1 = int(lines[3])
        end1 = int(lines[4])
        idx1 = int(lines[5])
        imagi_count = 100
        while imagi_count > 0:
            chrom2 = np.random.randint(22) + 1
            chr2 = chrom2
            
            seq2 = chrom2seq[f'chr{chrom2}']
    
            pos2 = np.random.randint(len(seq2)-10000) + 10000
            s2 = seq2[pos2:pos2+101]
            if 'NNNNNNN' in s2:
                continue
            pos2 = int((pos2 + 51)//resolution + 1)
            start2 = pos2-100
            end2 = pos2+100
            name2 = f'chr{chrom2:02d}_{pos2:04d}'
            idx2 = name2idx[name2]

            imagi_count = imgaic_np[idx1][idx2]
            hic_count = hic_np[idx1][idx2]

            atac1 = atac_np[idx1][0]
            atac2 = atac_np[idx2][0]
        fp.write(f'{seq1}\t{s2}\t{chr1}\t{start1}\t{end1}\t{chr2}\t{start2}\t{end2}\t{imagi_count}\t{hic_count}\t{atac1}\t{atac2}\n')
        #fp.write(f'{seq1}\t{s2}\t{chr1}\t{chr2}\t{idx1}\t{idx2}\t{imagi_count}\t{hic_count}\t{atac1}\t{atac2}\n')